# DOWNLOAD THE TEXT CORPUS

In [1]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.gz
!gzip -d text8.gz -f

--2021-08-19 04:47:56--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.24
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.gz’

text8.gz            100%[===================>]  29.89M   874KB/s    in 35s     

2021-08-19 04:48:32 (863 KB/s) - ‘text8.gz’ saved [31344016/31344016]



# word2vec

## Train word2vec


In [2]:
!pip install gensim

In [3]:
from gensim import utils

with open('/content/text8','r') as file:
  documents = file.readlines()

class MyCorpus(object):
    def __iter__(self): 
        for line in documents:
            yield utils.simple_preprocess(line)

In [4]:
import gensim.models
sentences = MyCorpus()
CBoWModel = gensim.models.Word2Vec(sentences=sentences,
                                       sg = 0,    
                                       window = 10,
                                       size=100,
                                       min_count = 5,
                                       workers=16,
                                       iter = 18)

In [5]:
len(CBoWModel.wv.vocab.keys())

70965

In [6]:
# get vector for a word
CBoWModel.wv.get_vector('god')

array([-0.15399843,  0.09022599,  0.30945295, -0.07111701,  0.12193686,
       -0.00037566,  0.06300254,  0.07947888,  0.06284299, -0.03162529,
        0.13438408,  0.00337701,  0.12903976, -0.02058399, -0.12209886,
       -0.02763083, -0.08785211, -0.01072947, -0.0721125 ,  0.18494573,
        0.09609465,  0.03661953,  0.1065132 , -0.05657565,  0.11854696,
       -0.08992825,  0.0489223 ,  0.15656175,  0.12519139,  0.03536   ,
       -0.08771522,  0.03354008,  0.13986701,  0.09598158,  0.09977074,
        0.21826094, -0.20031042,  0.03265378, -0.27219924,  0.20722032,
       -0.27942374, -0.06308457,  0.00288361, -0.12157365,  0.03568288,
        0.03692308,  0.06353854,  0.20876816,  0.00214606, -0.15717643,
        0.09888441,  0.0209484 ,  0.05393324,  0.06295133, -0.08126391,
       -0.12099317,  0.01353336,  0.051792  , -0.03661415,  0.02351845,
       -0.06987565,  0.16171025, -0.01356991, -0.04619968, -0.09960846,
        0.10341901, -0.02509256,  0.06423491,  0.00074572, -0.05

In [7]:
CBoWModel.wv.most_similar('good',topn=4)

[('language', 0.9997063279151917),
 ('from', 0.9997049570083618),
 ('or', 0.9997001886367798),
 ('more', 0.999699592590332)]

## Pretrained Word2Vec Model

Pretrained word2vec model is released by Google. It is trained on Google News corpus. 

In [8]:
import gensim.downloader as api

model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


## Word Similarity

In [9]:
!pip install sklearn

In [11]:
import pandas as pd
import csv
data=pd.read_csv('/content/wordsim_similarity_goldstandard.txt',sep='\t')
data=pd.DataFrame(data)
data.columns=['Word1','Word2','RSS']
data.head(20)

,Word1,Word2,RSS
0,tiger,tiger,10.00
1,plane,car,5.77
2,train,car,6.31
3,television,radio,6.77
4,media,radio,7.42
5,bread,butter,6.19
6,cucumber,potato,5.92
7,doctor,nurse,7.00
8,professor,doctor,6.62
9,student,professor,6.81


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def get_word_similarity(word_1,word_2,model):
  
    word_1_vector = model.get_vector(word_1)
    word_2_vector = model.get_vector(word_2)
    similarity = cosine_similarity(word_1_vector.reshape(1,-1),word_2_vector.reshape(1,-1))[0][0]
    return similarity
for index,row in data.iterrows(): 
  data['CSS']=(get_word_similarity(row['Word1'],row['Word2'],model)) #CSS= COSINE SIMILARITY SCORE
data.head()

,Word1,Word2,RSS,CSS
0,tiger,tiger,10.00,0.118132
1,plane,car,5.77,0.118132
2,train,car,6.31,0.118132
3,television,radio,6.77,0.118132
4,media,radio,7.42,0.118132


## Spearman Correlation

In [20]:
from scipy import stats

L=data['CSS']
R=data['RSS']

for index,row in data.iterrows(): 
  p = stats.spearmanr(L,R)
  print(p)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrR

## fastText 

## Train fastText

In [18]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.9 MB/s 
  Using cached pybind11-2.7.1-py2.py3-none-any.whl (200 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3095925 sha256=78d4e1d064a7b3880b0e0324e89f6ef0d07346a4b1d37b8919eba9d721e65c1f
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [19]:
import fasttext
fassttext_model = fasttext.train_unsupervised(input = '/content/text8',
                                              ws = 5,
                                              model = 'cbow',
                                              dim = 100,
                                              epoch = 6)

## Word Similarity

In [22]:
for index,row in data.iterrows(): 
  data['CSS2']=get_word_similarity(row['Word1'],row['Word2'],model)
data.head(20)

,Word1,Word2,RSS,CSS,CSS2
0,tiger,tiger,10.00,0.118132,0.118132
1,plane,car,5.77,0.118132,0.118132
2,train,car,6.31,0.118132,0.118132
3,television,radio,6.77,0.118132,0.118132
4,media,radio,7.42,0.118132,0.118132
5,bread,butter,6.19,0.118132,0.118132
6,cucumber,potato,5.92,0.118132,0.118132
7,doctor,nurse,7.00,0.118132,0.118132
8,professor,doctor,6.62,0.118132,0.118132
9,student,professor,6.81,0.118132,0.118132


## Spearman Correlation

In [25]:
CSS=data['CSS2']
RSS=data['RSS']

for index,row in data.iterrows(): 
  p = stats.spearmanr(CSS,RSS)
  print(p)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrResult(correlation=nan, pvalue=nan)
SpearmanrR